In [1]:
import pandas as pd
import yaml
from typing import Text
from datetime import datetime
import numpy as np

import json
from geopy import distance

import warnings
warnings.filterwarnings("ignore")

In [62]:
config_path = "../config/params.yml"
config = yaml.load(open(config_path,  encoding="utf-8"), Loader=yaml.FullLoader)


preproc = config["preprocessing"]
training = config["train"]

# Препроцессинг

In [3]:
def get_dataset(dataset_path: Text) -> pd.DataFrame:
    """
    Получение данных по заданному пути
    :param dataset_path: путь до данных
    :return: датасет
    """
    return pd.read_csv(dataset_path)

In [4]:
df = get_dataset(preproc["cian_path"])

In [5]:
df.iloc[1]

Название                                Многокомнатная квартира, 190 м²
Адрес                 Москва, ЦАО, р-н Тверской, ул. Большая Дмитров...
Метро                                                         Чеховская
Время до метро                                                   7 мин.
Цена                                                         89000000.0
Цена за квадрат                                                468421.0
Общая площадь                                                       190
Жилая площадь                                                       126
Площадь кухни                                                        22
Этаж                                                             3 из 5
Год постройки                                                    1912.0
Тип жилья                                                      Вторичка
Высота потолков                                                   3,6 м
Санузел                                                   3 совм

In [6]:
df.shape

(10307, 30)

## Общий анализ данных.

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10307 entries, 0 to 10306
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Название            10291 non-null  object 
 1   Адрес               10291 non-null  object 
 2   Метро               10289 non-null  object 
 3   Время до метро      10253 non-null  object 
 4   Цена                10291 non-null  float64
 5   Цена за квадрат     10291 non-null  float64
 6   Общая площадь       10291 non-null  object 
 7   Жилая площадь       7882 non-null   object 
 8   Площадь кухни       8477 non-null   object 
 9   Этаж                10291 non-null  object 
 10  Год постройки       8319 non-null   float64
 11  Тип жилья           10291 non-null  object 
 12  Высота потолков     7879 non-null   object 
 13  Санузел             9021 non-null   object 
 14  Вид из окон         7838 non-null   object 
 15  Ремонт              8428 non-null   object 
 16  Стро

In [8]:
print(f"Доля пропусков:\n{round(df.isna().sum() / df.shape[0] * 100, 2)}")

Доля пропусков:
Название               0.16
Адрес                  0.16
Метро                  0.17
Время до метро         0.52
Цена                   0.16
Цена за квадрат        0.16
Общая площадь          0.16
Жилая площадь         23.53
Площадь кухни         17.75
Этаж                   0.16
Год постройки         19.29
Тип жилья              0.16
Высота потолков       23.56
Санузел               12.48
Вид из окон           23.95
Ремонт                18.23
Строительная серия    42.23
Мусоропровод          55.68
Количество лифтов     22.83
Тип дома              20.07
Тип перекрытий        25.66
Парковка              40.66
Подъезды              30.90
Отопление             25.66
Аварийность           25.66
Газоснабжение         72.83
Балкон/лоджия         48.97
Год сдачи             85.13
Дом                   90.03
Отделка               86.25
dtype: float64


<br>
Посмотрю на значения признаков у которых больше 70% пропусков в данных.
<br><br>

In [9]:
isna_list = (df.isna().sum() / df.shape[0] > 0.7).loc[lambda x: x == True].index

In [10]:
for col in isna_list:
    print(f"{col} : {df[col].value_counts(normalize=True, dropna=False)*100}\n")

Газоснабжение : Газоснабжение
NaN            72.833996
Центральное    27.020472
Автономное      0.145532
Name: proportion, dtype: float64

Год сдачи : Год сдачи
NaN       85.126613
2023.0     4.686136
2024.0     4.104007
2025.0     3.094984
2026.0     1.348598
2021.0     0.407490
2022.0     0.388086
2027.0     0.388086
2013.0     0.135830
2019.0     0.106724
2020.0     0.097021
2016.0     0.077617
2017.0     0.029106
2018.0     0.009702
Name: proportion, dtype: float64

Дом : Дом
NaN        90.026196
Не сдан     8.169205
Сдан        1.804599
Name: proportion, dtype: float64

Отделка : Отделка
NaN                   86.252062
Без отделки            8.101290
Чистовая               3.774134
Предчистовая           0.766469
Черновая               0.407490
Неизвестно             0.397788
С отделкой             0.242554
Под ключ               0.038809
Чистовая с мебелью     0.019404
Name: proportion, dtype: float64



В признаке "Газоснабжение" нет значения "без газоснабжения".Пустые значения говорят об отсутствии газа в доме, либо об отсутствии информации на сайте об этом доме. Так как у нас всего 2 значения, и "автономное" попадается только в 0.1%, то решаю этот признак удалить.  <br><br>
В признаке "Год сдачи" отсутствуют значения <2013 года. Пустые значения могут как раз обозначать этот период сдачи и отсутствие информации на сайте.
Так как у нас более менее хорошо заполненный признак "год постройки" то думаю что "год сдачи" тоже можно удалить<br><br>
В признаке "Дом" большинство пропусков могут принадлежать старым домам, которые по определению давно сданы. Так как пропусков почти 90% то удалю этот признак.<br><br>
В признаке "Отделка" скорее всего пропуски также относятся к старым домам. Здесь больше значений чем в предыдущих признаках, которые могут повлиять на цену, в дальнейшем заменю пустые значения на "неизвестно".

In [11]:
drop_lst = df[["Строительная серия", "Аварийность", "Подъезды"]]

In [12]:
for col in drop_lst:
    print(f"{col} : {df[col].value_counts(normalize=True, dropna=False)*100}\n")

Строительная серия : Строительная серия
NaN                      42.233434
Индивидуальный проект    34.675463
II-49                     3.550985
П-44                      2.454642
II-18                     2.318812
                           ...    
1МГ-601-Д                 0.009702
К-7                       0.009702
II-18-02/12               0.009702
II-17                     0.009702
ГМС-1                     0.009702
Name: proportion, Length: 66, dtype: float64

Аварийность : Аварийность
Нет    74.289318
NaN    25.662171
Да      0.048511
Name: proportion, dtype: float64

Подъезды : Подъезды
NaN      30.901329
1.0      13.747938
4.0      10.633550
3.0       8.877462
2.0       8.528185
6.0       6.034734
5.0       5.229456
8.0       3.735326
7.0       2.571068
10.0      2.008344
12.0      1.775492
9.0       1.688173
11.0      0.912002
14.0      0.640342
13.0      0.601533
15.0      0.504511
16.0      0.310469
18.0      0.291064
17.0      0.261958
21.0      0.242554
24.0      0.194043

<br>
Удалю строки в которых есть пропуски в признаках: Цена, Метро. Так как цена это целевой признак, а метро так как это важный признак в котором по моему мнению не допустимо заполнение приблизительными значениями.<br> 
Так же удалю признаки [ "Строительная серия","Аварийность", "Подъезды"] так как логически они мало влияют на цену либо в них сильно преобладает одно значение и пропуски.
<br> 
<br>

In [13]:
df = df.drop(
    columns=[
        "Газоснабжение",
        "Год сдачи",
        "Дом",
        "Строительная серия",
        "Аварийность",
        "Подъезды",
    ]
).copy()

In [14]:
df = df.drop(df["Цена"].isna().loc[lambda x: x == True].index)

In [15]:
df_clean = df.drop(df["Метро"].isna().loc[lambda x: x == True].index).reset_index(
    drop=True
)

In [16]:
print(f"Доля пропусков:\n{round(df_clean.isna().sum() / df_clean.shape[0] * 100, 2)}")

Доля пропусков:
Название              0.00
Адрес                 0.00
Метро                 0.00
Время до метро        0.35
Цена                  0.00
Цена за квадрат       0.00
Общая площадь         0.00
Жилая площадь        23.39
Площадь кухни        17.61
Этаж                  0.00
Год постройки        19.15
Тип жилья             0.00
Высота потолков      23.44
Санузел              12.34
Вид из окон          23.82
Ремонт               18.09
Мусоропровод         55.60
Количество лифтов    22.71
Тип дома             19.93
Тип перекрытий       25.53
Парковка             40.56
Отопление            25.53
Балкон/лоджия        48.88
Отделка              86.25
dtype: float64


In [17]:
round(df_clean.describe(), 2)

,Цена,Цена за квадрат,Год постройки
count,1.028900e+04,10289.00,8319.00
mean,5.063152e+07,498353.29,1982.39
std,1.168605e+08,391735.72,33.08
min,1.600000e+06,73648.00,1822.00
25%,1.340000e+07,292809.00,1963.00
50%,1.990000e+07,372549.00,1980.00
75%,3.950000e+07,525000.00,2014.00
max,3.488667e+09,4998082.00,2023.00


In [18]:
df_clean.describe(include=["object", "bool"])

,Название,Адрес,Метро,Время до метро,Общая площадь,Жилая площадь,Площадь кухни,Этаж,Тип жилья,Высота потолков,...,Вид из окон,Ремонт,Мусоропровод,Количество лифтов,Тип дома,Тип перекрытий,Парковка,Отопление,Балкон/лоджия,Отделка
count,10289,10289,10289,10253,10289,7882,8477,10289,10289,7877,...,7838,8428,4568,7952,8238,7662,6116,7662,5260,1415
unique,3550,5721,291,30,1954,899,391,967,6,131,...,3,4,2,42,7,4,4,7,19,8
top,"2-комн. квартира, 45 м²","Москва, ЦАО, р-н Арбат, Поварская ул., 8/1к1",Арбатская,6 мин.,45,20,6,2 из 5,Вторичка,"2,64 м",...,Во двор,Косметический,Да,1 пассажирский,Монолитный,Железобетонные,Наземная,Центральное,1 балкон,Без отделки
freq,117,32,143,858,144,214,814,219,7590,1479,...,3918,2706,3599,2433,3524,5545,3302,6282,2500,833


## Жилая площадь, Площадь кухни, Общая площадь, Высота потолков.

<br>
Заменю все запятые на точки и удалю лишние пробелы.
<br>
<br>

In [19]:
def replace_comma(
    data: pd.DataFrame, lst: list, inplace_true: bool = False
) -> pd.DataFrame:
    """
    Замена запятых на точки в столбцах и удаление лишних символов (\xa0)
    :param data: дата фрейм
    :param lst: список из названий колонок
    :param inplace_true: True - изменения сразу применятся к data, Fasle - создается новый дата фрейм
    """
    data_new = data.copy()
    data_new[lst] = (
        data[lst]
        .apply(lambda x: x.str.split("\xa0").str[0].str.replace(",", ".").astype(float))
        .copy()
    )
    if inplace_true == True:
        data = data_new.copy()
        return data
    else:
        return data_new

In [20]:
df_clean = replace_comma(
    df_clean, ["Жилая площадь", "Площадь кухни", "Общая площадь", "Высота потолков"]
).copy()

## Время до метро.

Удалю лишние символы.

In [21]:
df_clean["Время до метро"] = df_clean["Время до метро"].str.split(" ").str[0].copy()

In [22]:
df_clean["Время до метро"] = pd.to_numeric(df_clean["Время до метро"], errors="coerce")

## Санузел, Количество лифтов, Балкон/лоджия

Удалим тип санузлов, лифтов и балконов и оставим их количество, чтобы избавиться от большого кол-ва уникальных значений. И преобразуем признаки в числовой тип.

In [23]:
def sum_obj(obj: pd.Series) -> pd.Series:
    """
    Удаление лишней информации из значений признака и преобразование в числовой тип.
    :param obj: колонка из дата фрейма.
    """

    temp_obj_1 = pd.to_numeric(obj.str.split(" ", expand=True)[0], errors="coerce")
    temp_obj_2 = pd.to_numeric(obj.str.split(" ", expand=True)[2], errors="coerce")
    temp = pd.concat([temp_obj_1, temp_obj_2], axis=1)
    temp = temp.sum(axis=1, min_count=1).astype("Int64")
    return temp

In [24]:
df_clean[["Санузел", "Количество лифтов", "Балкон/лоджия"]] = df_clean[
    ["Санузел", "Количество лифтов", "Балкон/лоджия"]
].apply(sum_obj)

## Этаж

Разделим этажи на 2 признака: Этаж и Этажность здания.

In [25]:
floor = df_clean["Этаж"].str.split(" из ", expand=True)

In [26]:
df_clean["Этаж"] = floor[0]

In [27]:
df_clean["Этажность здания"] = floor[1]

In [28]:
df_clean["Этаж"] = pd.to_numeric(df_clean["Этаж"], errors="coerce")

## Кол-во комнат

Добавим признак "Кол-во комнат". И так как нас интересуют квартиры с кол-вом комнат до 5 включительно для постоянного проживания, то удалим квартиры с большим кол-вом комнат и апартаменты.

In [29]:
df_clean["Кол-во комнат"] = (
    df_clean["Название"].str.split(" ", expand=True)[0].str.split("-", expand=True)[0]
)

In [30]:
df_clean["Кол-во комнат"].value_counts(normalize=True)

Кол-во комнат
2                 0.327729
3                 0.268150
1                 0.216834
4                 0.083584
Апартаменты       0.030810
5                 0.029546
Студия,           0.028671
Многокомнатная    0.012732
Многокомнатные    0.001944
Name: proportion, dtype: float64

In [31]:
def drop_rooms(data: pd.DataFrame, col: str) -> pd.Series:
    """
    Удаление лишних уникальных значений в признаке 'Кол-во комнат', квартиры с большим кол-вом комнат и апартаменты.
    :param data: Датасет
    :param col: Название колонки "кол-во комнат"
    """
    to_drop = (
        data[col]
        .loc[
            lambda x: (x == "Многокомнатная")
            | (x == "Многокомнатные")
            | (x == "Апартаменты")
        ]
        .index
    )
    data_new = data.drop(index=to_drop).copy()
    data_new[col][data_new[col] == "Студия,"] = 1
    data_new[col] = pd.to_numeric(data_new[col], errors="coerce")
    return data_new

In [32]:
df_clean = drop_rooms(df_clean, "Кол-во комнат")

In [33]:
df_clean =df_clean.drop(columns=['Название']) 

## Тип Жилья

Уберем лишнюю информацию для снижения количества уникальных значений.

In [34]:
df_clean["Тип жилья"].value_counts()

Тип жилья
Вторичка                   7478
Новостройка                1281
Вторичка Апартаменты        814
Новостройка Апартаменты     194
Вторичка Пентхаус            29
Новостройка Пентхаус         25
Name: count, dtype: int64

In [35]:
df_clean["Тип жилья"] = df_clean["Тип жилья"].str.split(" ").str[0]

## Заполнение пропусков.

In [36]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9821 entries, 0 to 10288
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Адрес              9821 non-null   object 
 1   Метро              9821 non-null   object 
 2   Время до метро     9792 non-null   float64
 3   Цена               9821 non-null   float64
 4   Цена за квадрат    9821 non-null   float64
 5   Общая площадь      9821 non-null   float64
 6   Жилая площадь      7590 non-null   float64
 7   Площадь кухни      8213 non-null   float64
 8   Этаж               9821 non-null   int64  
 9   Год постройки      8000 non-null   float64
 10  Тип жилья          9821 non-null   object 
 11  Высота потолков    7563 non-null   float64
 12  Санузел            8609 non-null   Int64  
 13  Вид из окон        7486 non-null   object 
 14  Ремонт             8004 non-null   object 
 15  Мусоропровод       4455 non-null   object 
 16  Количество лифтов  7695 non-

### Время до метро

In [37]:
df_clean["Время до метро"] = df_clean["Время до метро"].fillna(
        df_clean["Время до метро"].median()
    )
df_clean["Время до метро"] = df_clean["Время до метро"].copy().astype(int)

### Жилая площадь, Площадь кухни

Расчитаем медианное значение соотношения площадей жилых и кухни к общей площади и заполним ими пропуски.


In [38]:
def ar_part(data: pd.DataFrame, col: str, main_col: str) -> pd.Series:
    """
    Заполнение пропусков в зависимости от отношения площади признака от общей площади.
    :param data: датасет
    :param col: Название колонки площади необходимого признака.
    :param main_col: Название колонки общей площади.
    """
    part = round((data[col] / data[main_col]).median(), 2)
    col_new = data[col].fillna(data[main_col] * part)
    return col_new

In [39]:
for i in df_clean[["Жилая площадь", "Площадь кухни"]]:
    df_clean[i] = ar_part(df_clean, i, "Общая площадь")

Расчитаем медианное значение соотношения площади кухни к общей площади и заполним им пропуски.


### Год постройки

Все пропуски обозначают то что дом еще не сдан, и будет сдан либо в текущем году либо в следующем. Заполню пропуски следющим годом.

In [40]:
currentYear = datetime.now().year

In [41]:
df_clean["Год постройки"] = (
    df_clean["Год постройки"].fillna(currentYear + 1).astype(int)
)

### Санузел , Балкон/лоджия.

In [42]:
def replace_group(data: pd.DataFrame, main_col: str, col_1: str, col_2: str):
    """
    Заполнение модой через groupby.
    :param data: Дата фрейм
    :param main_col: Название колонки по которой идет группировка
    :param col_1: Название колонки в которой происходит заполнение пустых значений
    :param col_2: Название колонки в которой происходит заполнение пустых значений
    """
    for col in data[[col_1, col_2]]:
        data[col] = data[col].fillna(
            data.groupby(main_col)[col].transform(lambda x: x.mode()[0])
        )
        data[col] = data[col].astype(int)
    data_new = data.copy()
    return data_new

In [43]:
df_clean = replace_group(df_clean, "Кол-во комнат", "Санузел", "Балкон/лоджия").copy()

### Количество лифтов

In [44]:
def replace_elev(row: pd.Series) -> int:
    """
    Заполнение пропусков в признаке кол-во лифтов в зависимости от этажности здания согласно СНиП 31-01-2003
    :param row: строка дата фрейма.
    """

    if row["Количество лифтов"] == 0:
        floors = int(row["Этажность здания"])
        if floors < 6:
            return 0
        elif floors < 10:
            return 1
        elif floors < 20:
            return 2
        elif floors < 25:
            return 3
        else:
            return 4
    else:
        return row["Количество лифтов"]

In [45]:
df_clean["Количество лифтов"].fillna(0, inplace=True)

In [46]:
df_clean["Количество лифтов"] = df_clean["Количество лифтов"].astype(int)

In [47]:
df_clean["Этажность здания"] = df_clean["Этажность здания"].astype(int)

In [48]:
df_clean["Количество лифтов"] = df_clean.apply(lambda row: replace_elev(row), axis=1)

### Высота потолков.

Заполню медианным значением так как есть выбросы.

In [49]:
df_clean["Высота потолков"] = df_clean["Высота потолков"].fillna(
    df_clean["Высота потолков"].median()
)

### Отделка.

Здесь много пропусков, и вообще отделка больше относится к новостройкам. Но также есть и вторички, которые продают с голыми стенами, соответственно, отделка здесь уже будет играть роль. Заменю пропуски на "Неизвестно".

In [50]:
df_clean["Отделка"].value_counts()

Отделка
Без отделки           812
Чистовая              375
Предчистовая           77
Черновая               42
Неизвестно             41
С отделкой             25
Под ключ                4
Чистовая с мебелью      2
Name: count, dtype: int64

In [51]:
df_clean["Отделка"] = df_clean["Отделка"].fillna("Неизвестно")

### Другие категориальные признаки.

Заполню остальные категориальные признаки модой.

In [52]:
to_fill = df_clean[
    [
        "Отопление",
        "Вид из окон",
        "Ремонт",
        "Тип дома",
        "Тип перекрытий",
        "Парковка",
        "Мусоропровод",
    ]
]

In [53]:
for column in to_fill:
    df_clean[column] = df_clean[column].fillna(df_clean[column].mode()[0])

In [54]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9821 entries, 0 to 10288
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Адрес              9821 non-null   object 
 1   Метро              9821 non-null   object 
 2   Время до метро     9821 non-null   int32  
 3   Цена               9821 non-null   float64
 4   Цена за квадрат    9821 non-null   float64
 5   Общая площадь      9821 non-null   float64
 6   Жилая площадь      9821 non-null   float64
 7   Площадь кухни      9821 non-null   float64
 8   Этаж               9821 non-null   int64  
 9   Год постройки      9821 non-null   int32  
 10  Тип жилья          9821 non-null   object 
 11  Высота потолков    9821 non-null   float64
 12  Санузел            9821 non-null   int32  
 13  Вид из окон        9821 non-null   object 
 14  Ремонт             9821 non-null   object 
 15  Мусоропровод       9821 non-null   object 
 16  Количество лифтов  9821 non-

In [55]:
df_clean.describe()

,Время до метро,Цена,Цена за квадрат,Общая площадь,Жилая площадь,Площадь кухни,Этаж,Год постройки,Высота потолков,Санузел,Количество лифтов,Балкон/лоджия,Этажность здания,Кол-во комнат
count,9821.000000,9.821000e+03,9.821000e+03,9821.000000,9821.000000,9821.000000,9821.000000,9821.000000,9821.000000,9821.000000,9821.000000,9821.000000,9821.000000,9821.000000
mean,9.411567,4.659237e+07,4.927128e+05,72.745768,42.658307,12.481586,8.189594,1990.434579,2.936179,1.356888,2.020568,1.102230,15.777314,2.291722
std,5.222118,9.029864e+07,3.816172e+05,51.422336,31.805155,8.809804,7.782657,33.306659,0.627947,0.682939,1.615727,0.365988,11.757963,1.040191
min,1.000000,2.390000e+06,7.364800e+04,11.000000,3.000000,1.000000,-1.000000,1822.000000,0.000000,1.000000,0.000000,1.000000,2.000000,1.000000
25%,5.000000,1.360000e+07,2.916670e+05,42.000000,23.000000,7.000000,3.000000,1967.000000,2.700000,1.000000,1.000000,1.000000,8.000000,1.000000
50%,9.000000,2.000000e+07,3.705880e+05,58.000000,33.700000,10.000000,6.000000,2000.000000,2.870000,1.000000,2.000000,1.000000,12.000000,2.000000
75%,13.000000,3.890000e+07,5.206400e+05,82.800000,50.000000,15.000000,11.000000,2022.000000,3.000000,2.000000,3.000000,1.000000,20.000000,3.000000
max,30.000000,1.910410e+09,4.998082e+06,705.000000,450.000000,136.500000,82.000000,2024.000000,31.000000,7.000000,28.000000,6.000000,95.000000,5.000000


Удалим аномалии в данных: все отрицательные значения и значения меньше 2 в "Высота потолков".

In [56]:
def anomaly_drop(data: pd.DataFrame) -> pd.DataFrame:
    """Удаление аномалий в данных: Отрицательные значения и значения меньше 2 в "Высота потолков"
    :param data: Датасет
    """
    df_num = data.select_dtypes(include=np.number)
    drop_neg = []
    for i, j in df_num.iterrows():
        for k in j:
            if k < 0:
                drop_neg.append(i)
    drop_neg += data[data["Высота потолков"] < 2].index.tolist()
    df_fin = data.drop(index=drop_neg)
    df_fin = df_fin.reset_index(drop=True)
    return df_fin

In [57]:
df_fin = anomaly_drop(df_clean)

In [58]:
df_fin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9818 entries, 0 to 9817
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Адрес              9818 non-null   object 
 1   Метро              9818 non-null   object 
 2   Время до метро     9818 non-null   int32  
 3   Цена               9818 non-null   float64
 4   Цена за квадрат    9818 non-null   float64
 5   Общая площадь      9818 non-null   float64
 6   Жилая площадь      9818 non-null   float64
 7   Площадь кухни      9818 non-null   float64
 8   Этаж               9818 non-null   int64  
 9   Год постройки      9818 non-null   int32  
 10  Тип жилья          9818 non-null   object 
 11  Высота потолков    9818 non-null   float64
 12  Санузел            9818 non-null   int32  
 13  Вид из окон        9818 non-null   object 
 14  Ремонт             9818 non-null   object 
 15  Мусоропровод       9818 non-null   object 
 16  Количество лифтов  9818 

In [59]:
def save_unique_train_data(
    data: pd.DataFrame, drop_columns: list, unique_values_path: str
) -> None:
    """
    Сохранение словаря с признаками и уникальными значениями
    :param drop_columns: список с признаками для удаления
    :param data: датасет
    :param unique_values_path: путь до файла со словарем
    :return: None
    """
    unique_df = data.drop(columns=drop_columns, axis=1)
    # создаем словарь с уникальными значениями для вывода в UI
    dict_unique = {key: unique_df[key].unique().tolist() for key in unique_df.columns}
    with open(unique_values_path, "w") as file:
        json.dump(dict_unique, file)

In [63]:
save_unique_train_data(
            data=df_fin,
            drop_columns=training["drop_columns_unique"],
            unique_values_path=training["unique_values_path"],
        )

In [64]:
with open(training["unique_values_path"]) as json_file:
        unique_values = json.load(json_file)

In [69]:
type(unique_values['Время до метро'])

list

Заполнены все путые значения кроме время до метро. Его заполню дальше после добавления геоданных.

In [63]:
df_fin.describe()

,Время до метро,Цена,Цена за квадрат,Общая площадь,Жилая площадь,Площадь кухни,Этаж,Год постройки,Высота потолков,Санузел,Количество лифтов,Балкон/лоджия,Этажность здания,Кол-во комнат
count,9818.000000,9.818000e+03,9.818000e+03,9818.000000,9818.000000,9818.000000,9818.000000,9818.000000,9818.000000,9818.000000,9818.000000,9818.000000,9818.000000,9818.000000
mean,9.411795,4.659778e+07,4.927652e+05,72.742767,42.655024,12.482345,8.190874,1990.452944,2.936784,1.356895,2.020982,1.102261,15.779690,2.291505
std,5.222074,9.031080e+07,3.816471e+05,51.427789,31.807988,8.810997,7.783096,33.291813,0.626642,0.682994,1.615780,0.366039,11.758861,1.040178
min,1.000000,2.390000e+06,7.364800e+04,11.000000,3.000000,1.000000,1.000000,1822.000000,2.000000,1.000000,0.000000,1.000000,2.000000,1.000000
25%,5.000000,1.360000e+07,2.917870e+05,42.000000,23.000000,7.000000,3.000000,1967.000000,2.700000,1.000000,1.000000,1.000000,8.000000,1.000000
50%,9.000000,2.000000e+07,3.705880e+05,58.000000,33.700000,10.000000,6.000000,2000.000000,2.870000,1.000000,2.000000,1.000000,12.000000,2.000000
75%,13.000000,3.890000e+07,5.206170e+05,82.700000,50.000000,15.000000,11.000000,2022.000000,3.000000,2.000000,3.000000,1.000000,20.000000,3.000000
max,30.000000,1.910410e+09,4.998082e+06,705.000000,450.000000,136.500000,82.000000,2024.000000,31.000000,7.000000,28.000000,6.000000,95.000000,5.000000


In [64]:
df_fin.describe(include=["bool", "object"])

,Адрес,Метро,Тип жилья,Вид из окон,Ремонт,Мусоропровод,Тип дома,Тип перекрытий,Парковка,Отопление,Отделка
count,9818,9818,9818,9818,9818,9818,9818,9818,9818,9818,9818
unique,5588,290,2,3,4,2,7,4,4,7,8
top,"Москва, ЦАО, р-н Арбат, Поварская ул., 8/1к1",Арбатская,Вторичка,Во двор,Косметический,Да,Монолитный,Железобетонные,Наземная,Центральное,Неизвестно
freq,31,135,8318,6112,4474,8916,5346,7820,7121,8530,8481


In [65]:
def create_data(data: pd.DataFrame, dataset_path: Text):
    """
    Создание csv файла с данными.
    :param data: датасет.
    :param dataset_path: путь к создаваемому файлу.
    """
    data.to_csv(dataset_path, index=False)

In [66]:
create_data(data=df_fin, dataset_path=preproc["cian_clean_path"])

In [67]:
df_fin.iloc[1]

Адрес                Москва, ЦАО, р-н Тверской, Тверская ул., 12С7
Метро                                                   Пушкинская
Время до метро                                                   3
Цена                                                    75000000.0
Цена за квадрат                                           375000.0
Общая площадь                                                200.0
Жилая площадь                                                120.0
Площадь кухни                                                 20.0
Этаж                                                             6
Год постройки                                                 1917
Тип жилья                                                 Вторичка
Высота потолков                                                3.8
Санузел                                                          4
Вид из окон                                        На улицу и двор
Ремонт                                               Косметиче

# Геокодирование

## Поиск и добавление координат

### Поиск координат метро.

In [68]:
df_fin["Город и метро"] = "Москва, метро " + df_fin["Метро"]
df_fin["Город и метро"].head()

0        Москва, метро Трубная
1     Москва, метро Пушкинская
2    Москва, метро Охотный ряд
3    Москва, метро Охотный ряд
4    Москва, метро Театральная
Name: Город и метро, dtype: object

In [69]:
df_fin["Город и метро"].nunique()

290

In [70]:
def metro_point(data: pd.DataFrame, metro_path):
    """
    Поиск координат метро
    :param data: датасет.
    :return:
    """
    with open(metro_path) as file:
        metro = json.load(file)
    data["координаты метро"] = data["Город и метро"].map(metro)
    data["широта метро"] = data["координаты метро"].apply(lambda x: x[0])
    data["долгота метро"] = data["координаты метро"].apply(lambda x: x[1])

In [71]:
metro_point(df_fin, preproc["metro_path"])

### Поиск координат дома.

In [72]:
def address_point(data: pd.DataFrame, address_path):
    """
    Поиск координат дома
    :param data: датасет.
    :return:
    """
    with open(address_path) as file:
        address = json.load(file)
    data["координаты дома"] = data["Адрес"].map(address)
    data["широта дома"] = data["координаты дома"].apply(lambda x: x[0])
    data["долгота дома"] = data["координаты дома"].apply(lambda x: x[1])

In [73]:
address_point(df_fin, preproc["address_path"])

### Поиск расстояний до центра и метро.

In [74]:
def get_distance(data: pd.DataFrame, lon_center: float, lat_center: float):
    """
    Добавление дистанции до метро и до центра.
    :param lat_center: широта центра Москвы
    :param lon_center: долгота центра Москвы
    :param data: датасет
    :return:
    """
    data["Расстояние до центра"] = data[["широта дома", "долгота дома"]].apply(
        lambda x: distance.distance((x[0], x[1]), (lat_center, lon_center)).km, axis=1
    )
    data["Расстояние до метро"] = data[
        ["широта дома", "долгота дома", "широта метро", "долгота метро"]
    ].apply(lambda x: distance.distance((x[0], x[1]), (x[2], x[3])).km, axis=1)

In [75]:
get_distance(
    data=df_fin, lon_center=preproc["lon_center"], lat_center=preproc["lat_center"]
)

## Добавление признака "Округ"

In [76]:
def get_district(data: pd.DataFrame):
    """
    Добавление признака Округ.
    :param data: Датасет
    :return:
    """
    districts = [
        "ЦАО",
        "ЮАО",
        "ЮЗАО",
        "ЮВАО",
        "ЗАО",
        "СВАО",
        "ВАО",
        "САО",
        "СЗАО",
        "НАО (Новомосковский)",
        "ЗелАО",
    ]
    data["district"] = data["Адрес"].str.split(", ", expand=True)[1]

    district = dict()
    for i in range(0, len(data["Метро"])):
        if i not in district.keys():
            if data["district"][i] in districts:
                district[data["Метро"][i]] = data["district"][i]
    data["Округ"] = data["Метро"].map(district)

    for ind, dis in enumerate(data["Округ"]):
        if dis not in districts:
            data["Округ"][ind] = "Неизвестно"

    data["Округ"] = data["Округ"].str.split(" ", expand=True)[0]

In [77]:
get_district(df_fin)

In [78]:
df_fin["Округ"].value_counts()

Округ
ЦАО           2616
ЮАО           1439
ЮЗАО          1037
ЮВАО           948
ЗАО            824
ВАО            808
СВАО           800
САО            738
СЗАО           402
НАО            195
Неизвестно       6
ЗелАО            5
Name: count, dtype: int64

## Удаление лишних колонок

In [79]:
def removing_excess(data: pd.DataFrame) -> pd.DataFrame:
    """
    Удаление лишних колонок.
    :param data: Датасет
    :return: новый датасет
    """
    data_clean = data.drop(
        ["Город и метро", "координаты метро", "координаты дома", "district"], axis=1
    )
    return data_clean

In [80]:
df_full = removing_excess(df_fin)

In [81]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9818 entries, 0 to 9817
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Адрес                 9818 non-null   object 
 1   Метро                 9818 non-null   object 
 2   Время до метро        9818 non-null   int32  
 3   Цена                  9818 non-null   float64
 4   Цена за квадрат       9818 non-null   float64
 5   Общая площадь         9818 non-null   float64
 6   Жилая площадь         9818 non-null   float64
 7   Площадь кухни         9818 non-null   float64
 8   Этаж                  9818 non-null   int64  
 9   Год постройки         9818 non-null   int32  
 10  Тип жилья             9818 non-null   object 
 11  Высота потолков       9818 non-null   float64
 12  Санузел               9818 non-null   int32  
 13  Вид из окон           9818 non-null   object 
 14  Ремонт                9818 non-null   object 
 15  Мусоропровод         

In [82]:
create_data(data=df_full, dataset_path=preproc["cian_full_path"])